In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import BaseCrossValidator
from sklearn.linear_model import Ridge
import time
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor

In [6]:
df = pd.read_excel('Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833,12,0.21,2,22,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800,12,0.21,2,22,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826,12,0.21,2,22,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831,12,0.21,2,22,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834,12,0.21,2,22,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676,36,0.21,2,41,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683,36,0.21,2,42,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340900,50151.250000,77962.397730,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846,36,0.21,2,42,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694100,47515.850980,73948.509800,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876,36,0.21,2,42,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943200,42468.990340,65994.485800,3.789568e+07,3761.343750


In [7]:
df.replace('#DIV/0!',np.nan,inplace=True)
df.dropna(inplace=True)

C:\Users\TI 2023\AppData\Local\Temp\ipykernel_15244\1019318869.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('#DIV/0!',np.nan,inplace=True)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37527 entries, 0 to 37527
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Backhoe      37527 non-null  int64  
 1   HP(watt)     37527 non-null  float64
 2   Norm_MAP     37527 non-null  float64
 3   RPM          37527 non-null  float64
 4   Age          37527 non-null  int64  
 5   Load_Factor  37527 non-null  float64
 6   Engine_Tier  37527 non-null  int64  
 7   TEMP[C]      37527 non-null  float64
 8   Fuel[g/s]    37527 non-null  float64
 9   NOx[g/s]     37527 non-null  float64
 10  HC[g/s]      37527 non-null  float64
 11  CO[g/s]      37527 non-null  float64
 12  CO2[g/s]     37527 non-null  float64
 13  PM[mg/s]     37527 non-null  float64
 14  Nox (g/l)    37527 non-null  float64
 15  HC (g/l)     37527 non-null  float64
 16  CO (g/l)     37527 non-null  float64
 17  CO2 (g/l)    37527 non-null  float64
 18  PM (g/l)     37527 non-null  float64
 19  Fuel[g/hr

In [9]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['CO[g/s]']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

C:\Users\TI 2023\AppData\Local\Temp\ipykernel_15244\1236518287.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()
C:\Users\TI 2023\AppData\Local\Temp\ipykernel_15244\1236518287.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test = y_test.ravel()


In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(26268, 7)
(26268,)
(11259, 7)
(11259,)


In [12]:
rf = BaggingRegressor()

In [13]:
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

In [14]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

rmse_score = rmse(y_test, y_pred)
mape_score = mape(y_test, y_pred)

print(f'RMSE: {rmse_score:.2f}')
print(f"MAPE: {mape_score:.2f}%")

RMSE: 0.02
MAPE: 16079859622.43%


GRID

In [15]:
params = {
    'n_estimators': range(10, 500),
    'max_samples': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'max_features': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'bootstrap' : [True, False],
    'bootstrap_features': [True, False],
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error)

In [16]:
grid = GridSearchCV(
    rf,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE', 
    cv=2,  
    verbose=1,
    n_jobs=-1
)

start_time = time.time()
gbr_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 2 folds for each of 70560 candidates, totalling 141120 fits
Best: 0.020267 using {'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.5, 'max_samples': 0.8, 'n_estimators': 10}
Best RMSE: 0.020266747231686387
Best MAPE: 28248004746.211926
Execution time: 38728.51130080223 s


In [17]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 0.01811243196796255
MAPE: 27393365524.59%


In [18]:
pd.DataFrame(grid.cv_results_).to_excel('BR_GS_unscal_CO.xlsx', index=False)

random search

In [19]:
rs = RandomizedSearchCV(
    rf,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    n_iter=100,
    cv=3,  
    verbose=1,
    n_jobs=-1
)

start_time = time.time()
rs_result = rs.fit(X_train,y_train)

print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best: 0.020338 using {'n_estimators': 402, 'max_samples': 1.0, 'max_features': 1.0, 'bootstrap_features': False, 'bootstrap': False}
Best RMSE: 0.02033807865278493
Best MAPE: 14449598948.396544
Execution time: 133.12838673591614 s


In [20]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 0.02069839727675142
MAPE: 14148082412.00%


In [21]:
pd.DataFrame(rs.cv_results_).to_excel('BR_RS_unscal_CO.xlsx', index=False)

Bayessian

In [22]:
bootstraps = [True, False]
bootstrap_feature = [True, False]

def br_regression_cv(n_estimators, max_samples, max_features, bootstrap, bootstrap_features):
    rf = BaggingRegressor(
        n_estimators=int(n_estimators),
        max_samples=float(max_samples),
        max_features=float(max_features),
        bootstrap=bootstraps[int(round(bootstrap))],  
        bootstrap_features=bootstrap_feature[int(round(bootstrap_features))]
    )
    scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse 

In [23]:
bayes_opt = BayesianOptimization(
    br_regression_cv,
    {
        'n_estimators': (10, 500),
        'max_samples': (0.5,1),
        'max_features': (0.5, 1),
        'bootstrap': (0,1),
        'bootstrap_features':(0,1)
    }
)

In [24]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | bootstrap | bootst... | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------------------------------
| 1         | -0.01614  | 0.172     | 0.6717    | 0.6459    | 0.537     | 210.0     |
| 2         | -0.01596  | 0.1533    | 0.5571    | 0.6066    | 0.7667    | 106.3     |
| 3         | -0.01562  | 0.3805    | 0.8226    | 0.8496    | 0.6436    | 58.84     |
| 4         | -0.01579  | 0.9903    | 0.06086   | 0.9207    | 0.5689    | 348.1     |
| 5         | -0.01613  | 0.5248    | 0.4176    | 0.8056    | 0.8982    | 323.9     |
| 6         | -0.01573  | 0.3993    | 0.2083    | 0.9605    | 0.8681    | 454.2     |
| 7         | -0.01574  | 0.9066    | 0.7142    | 0.9016    | 0.6826    | 39.22     |
| 8         | -0.01676  | 0.2892    | 0.1566    | 0.6195    | 0.6172    | 495.6     |
| 9         | -0.01616  | 0.9288    | 0.2412    | 0.8142    | 0.5973    | 112.9     |
| 10        | -0.01746  | 0.8431    | 0.3492    | 0.51

In [25]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [26]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'bootstrap': 1.0, 'bootstrap_features': 1.0, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 48.80922374841956}


In [27]:
bootstraps = [True, False]
bootstrap_feature = [True, False]

best_bs = BaggingRegressor(
    n_estimators=int(best_params['n_estimators']),
    max_samples=float(best_params['max_samples']),
    max_features=float(best_params['max_features']),
    bootstrap=bootstraps[int(best_params['bootstrap'])],
    bootstrap_features=bootstrap_feature[int(best_params['bootstrap_features'])]
)
best_bs.fit(X_train, y_train)

BaggingRegressor(bootstrap=False, n_estimators=48)

In [28]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.020716249824325707
MAPE: 14150298829.73%
